In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2  #open cv
import mediapipe as mp   #mediapipe- pose estimation libs
import numpy as np       #trig
mp_drawing = mp.solutions.drawing_utils   #visualizing poses
mp_pose = mp.solutions.pose               #importing our pose estimation models 

In [ ]:
#CALCLUATE ANGLES
#between joints- USING MEDIAPIPE- TRIGO INVOLVED
def calculate_angle(a,b,c):
    a = np.array(a)  #first
    b = np.array(b)  #mid
    c = np.array(c)  #end
    
    radian = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radian*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [3]:
cap = cv2.VideoCapture(0)   #setting up webcam

#curl counter variables
counter = 0
stage = None

#set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    #loop through our feed
    while cap.isOpened():

        #current feed from webcam, ret is the return variable
        #frame gives us the image from the webcam
        ret, frame = cap.read()
        
        #detect stuff and render
        
        #recolor image to RGB- 
        #in RGB format when passing to openCV- 3 different color arrays
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False   #performance training
        
        #making detection
        #storing resukts in arrays
        results = pose.process(image)
        
        #recoloring it back to BGR, rerendering back for openCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #EXTRACT landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            #visualize- rendered angle to actual screen
            cv2.putText(image, str(angle),
                       tuple(np.multiply(elbow, [640,480]).astype(int)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            
            #curl counter logic
            
            #setting whether we're in down stage
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)
                        
        
        except:
            pass #steps out of loop 
        
        #Render curl counter
        #set up status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # reps data
        cv2.putText(image, 'REPS', (15,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA),
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        cv2.putText(image, 'STAGE', (65,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        
        #Render detections
        #draws utilities, passing- image, pose_landmarks=coordinates, pose connections ex- nose to right eye inner etc
        #with specifications of drawing landmark components
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #image that has landmarks
        cv2.imshow('Mediapipe Feed', image)

        #once we clear our feed- try to break out of the while loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()